<a href="https://colab.research.google.com/github/john-jiang-ca/MachineLearning/blob/master/ch13_exe10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
from tensorflow import keras
from keras.layers import Dense, Flatten
from pathlib import Path
from matplotlib import pyplot as plt
import pandas as pd
import math


from tensorflow.train import Example, Features, Feature, Int64List, BytesList

# Download data and split the paths into training, validate and test set

In [2]:
DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent / "aclImdb"
path


84131840/84125825 [==============================] - 6s 0us/step


PosixPath('/root/.keras/datasets/aclImdb')

In [3]:
def get_paths(path: Path):
  return [str(p) for p in path.glob("*.txt")]

train_neg = get_paths(path / "train" / "neg")
train_pos = get_paths(path / "train" / "pos")
test_neg = get_paths(path / "test" / "neg")
test_pos = get_paths(path / "test" / "pos")
val_neg = test_neg[:7500]
val_pos = test_pos[:7500]
test_neg = test_neg[7500:]
test_pos = test_pos[7500:]

# Create dataset that can be executed out core

In [4]:
def build_dataset_without_in_mem_fit(positive_paths, negative_paths):
  pos_set = tf.data.TextLineDataset(positive_paths, num_parallel_reads=10).map(lambda review: (review, 1))
  neg_set = tf.data.TextLineDataset(negative_paths, num_parallel_reads=10).map(lambda review: (review, 0))
  return tf.data.Dataset.concatenate(pos_set, neg_set)

In [5]:
training_set = build_dataset_without_in_mem_fit(train_pos, train_neg).shuffle(len(train_pos)*2).prefetch(1)
validation_set = build_dataset_without_in_mem_fit(val_pos, val_neg).shuffle(len(train_pos)*2).prefetch(1)
test_set = build_dataset_without_in_mem_fit(test_pos, test_neg).shuffle(len(train_pos)*2).prefetch(1)

## Explore what training set

*   Take some data from the training set
*   Create a pandas Datafraem using the data taken above
*  Display the data



In [6]:
demo_data = training_set.take(5)
data = []
for item in demo_data:
  review, label = item
  data.append({
  "review": review.numpy(),
  "label": label.numpy()
  })
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', -1)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


,review,label
0,"b'One star for the ""plot"". One star for the acting. One star for the dubbing into squeaky-voiced American. Five stars for Monica Broeke and Inge Maria Granzow, with their propensity for taking all their clothes off. And ten out of ten for the divine Emmanuelle B\xc3\xa9art, two years before she made \'Manon des sources\'. B\xc3\xa9art also undresses a couple of times, but even fully-clothed her presence is enough to make this film eminently watchable. Watch out for the scene where she tells her friend about the three ""first times"" for a girl. It\'s corny, but still far more erotic than the rather laughably choreographed ""love scenes"" featuring Broeke, Granzow and Patrick Bauchau. Incidentally, the cinematography is not great; the stills for the closing credits are a better indication of what David Hamilton is capable of.'",0
1,"b""I read James Hawes book. It was pretty neat, not great, but entertaining enough. Without having read the book I wouldn't have had the slightest idea what was going on, and it was still a stretch with that knowledge.<br /><br />Literally every element of this film is abysmal in ways I do not have the capacity to describe. Half digested fish could have made a better film with matchsticks and dayglo lipstick.<br /><br />Never before or since as a film made me feel so angry. The Mattress sequels came closest, but even they never reached such depths of utterly putrid nauseating appallingness that this bilge did.<br /><br />Since wasting 90 minutes of my life witnessing this plague on human kind I am now unable to even look at any book by James Hawes without feeling angry. That is the depth of hatred I have for this piece of sh*t. No, that's unfair. Let me apologise to all fecal matter for comparing you to the otherworldly evil that is Rancid Aluminium.<br /><br />Plain and simply a cancer on the world of cinema.""",0
2,"b'Like others, I have seen and studied most of the books and films concerning the Clutter Killings, including a few dramatic works thematically based on the actions and psycho-mythology of the participants to the crime -- including Capote himself. As to Capote, I cannot forgive him for willfully withholding Perry Smith\'s confessions, intimacies and writings from even the defense counsels. I believe truths and facts Capote ""reserved"" for his ""book,"" which required for Capote two guilty verdicts and capital punishment, would almost certainly have sustained a successful insanity defense for Perry Smith even under the old McNaughton Rule. Capote himself could never write another major literary work after ""In Cold Blood."" Shame and guilt. In my opinion, he willingly encouraged and planned the brutal capital punishment to provide the spectacular ending he required for his book/drama. To him, both men HAD to die for his book to succeed. The book had to justify itself by pretending it was about the horror of capital punishment. His actions and silence assured that ice-cold conclusion.<br /><br />Capote\'s book is not truth. It is not factual or journalistic. It is drama and melodrama spiced with his own creatively psychotic imagination. What most people consider the virtues of the contemporaneous first movie are stark images of Capote\'s mind, which may have been the most cold-blooded aspect of all. No wonder viewers ironically but necessarily prefer Blake\'s performance. That actor IS the nightmare from Capote\'s dishonest imaginings.<br /><br />So who is to say how the two killers should be played? Who is to judge what could make an essentially poetic psychotic snap from excessive courtesy and kindness to ""do it now"" killing? I agree with the few who see in Eric Roberts\' work a magnificent performance, Shakespearean in its range, yet played with heartbreaking sincerity. Anthony Edwards takes a much safer ""attitude mode"" to create a smarmy Hickok; but he is one-dimensional and boring, with only a few notes in his television range. Roberts is almost four-dimensio

## Create TFRecord dataset

In [7]:
def create_example(item):
  review, label = item
  return Example(
      features=Features(
          feature={
              "review": Feature(bytes_list = BytesList(value=[review.numpy()])),
              "label": Feature(int64_list = Int64List(value=[label]))
          }
      )
  )

In [11]:
def write_record(name, dataset, n_shards):
  paths = ["/content/drive/MyDrive/data/{}.tfrecord-{:02d}-of-{:02d}".format(name, index, n_shards) for index in range(n_shards)]
  writers = [tf.io.TFRecordWriter(path) for path in paths]
  print(paths)
  index = 0
  for item in dataset:
    writer = writers[index % n_shards]
    writer.write(create_example(item).SerializeToString())
    index = index+1

In [12]:
write_record("train-set", training_set, 5)

['/content/drive/MyDrive/data/train-set.tfrecord-00-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-01-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-02-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-03-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-04-of-05']


In [13]:
write_record("test-set", test_set, 2)
write_record("validate-set", validation_set, 2)

['/content/drive/MyDrive/data/test-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/test-set.tfrecord-01-of-02']
['/content/drive/MyDrive/data/validate-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/validate-set.tfrecord-01-of-02']


Define constants for paths

In [14]:
TRAIN_PATHS = ['/content/drive/MyDrive/data/train-set.tfrecord-00-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-01-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-02-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-03-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-04-of-05']
TEST_PATHS = ['/content/drive/MyDrive/data/test-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/test-set.tfrecord-01-of-02']
VALIDATE_PATHS = ['/content/drive/MyDrive/data/validate-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/validate-set.tfrecord-01-of-02']

In [15]:
def preprocess_tfrecord(record):
  feature_desc={
      "review": tf.io.FixedLenFeature([], tf.string, b""),
      "label": tf.io.FixedLenFeature([], tf.int64, -1)
  }
  example = tf.io.parse_single_example(record, feature_desc)
  return example['review'], example['label']

def get_tensor_dataset_from_path(paths):
  dataset = tf.data.TFRecordDataset(paths, num_parallel_reads=10)
  dataset = dataset.map(preprocess_tfrecord)
  return dataset.batch(32).prefetch(1)

In [16]:
TRAIN_SET = get_tensor_dataset_from_path(TRAIN_PATHS)

In [17]:
VAL_SET = get_tensor_dataset_from_path(VALIDATE_PATHS)
TST_SET = get_tensor_dataset_from_path(TEST_PATHS)

In [41]:

def text_pre_process(X_batch, total_slots=50):
    # shape = tf.shape(X_batch) * tf.constant([1, 0]) + tf.constant([0, total_slots])
    shape = tf.shape(X_batch) * tf.constant([1, 0]) + tf.constant([0, total_slots])
    Z = tf.strings.substr(X_batch, 0, 300)
    Z = tf.strings.lower(Z)
    Z = tf.strings.regex_replace(Z, b"<br\\s*/?>", b" ")
    Z = tf.strings.regex_replace(Z, b"[^a-z]", b" ")
    Z = tf.strings.split(Z)
    return Z.to_tensor(shape=shape, default_value=b"<pad>")

In [42]:
from collections import Counter

def get_vol(X: tf.Tensor, most_common: int):
  Y = X.numpy()
  ct = Counter()
  for review in Y:
    for word in review:
      ct[word]+=1
  return [wd for wd, count in ct.most_common(most_common)]


In [43]:
def extract_vol(X: tf.Tensor, max_words_per_review, max_vol_size):
  return get_vol(text_pre_process(X, max_words_per_review), max_vol_size)

In [60]:
text_example = tf.constant([
                            "this is an example", "I feel very good"
])

In [61]:
extract_vol(text_example, 100, 1000)

[b'<pad>', b'this', b'is', b'an', b'example', b'i', b'feel', b'very', b'good']

Need to specify the data type for Layer. The default type is `float`

In [62]:
class TextVectorization(keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__(dtype=tf.string, **kwargs)
  
  def fit(self, examples):
    words = extract_vol(examples, 100, 1000)
    size = len(words)
    word_ids = tf.range(size, dtype=tf.int64)
    self.total_size = size + math.ceil(size*0.1)
    vocab_init = tf.lookup.KeyValueTensorInitializer(tf.constant(words), word_ids)
    self.table = tf.lookup.StaticVocabularyTable(vocab_init, math.ceil(size * 0.1))
  
  def call(self, data):
    result = text_pre_process(data, 100)
    return self.table.lookup(result)

  def get_total_size(self):
    return self.total_size

In [63]:
class BagOfWords(keras.layers.Layer):
  def __init__(self, depth, **kwargs):
    self.depth = depth
    super().__init__(**kwargs)
  
  def call(self, data):
    one_hot_code = tf.one_hot(data, self.depth)
    return tf.reduce_sum(one_hot_code, axis=1)

In [64]:
bow = BagOfWords(tv.get_total_size()+1)

Get some samples 

In [65]:
sample = TRAIN_SET.take(50).map(lambda review, label: review)

In [66]:
concatenated_sample_reviews = np.concatenate(list(sample.as_numpy_iterator()), axis=0)

Create TextVencorization layer and bag of words layer

In [67]:
tv = TextVectorization(input_shape=[])
tv.fit(concatenated_sample_reviews)

In [68]:
bow = BagOfWords(tv.get_total_size())

In [69]:
tv(concatenated_sample_reviews)

<tf.Tensor: shape=(1600, 100), dtype=int64, numpy=
array([[   8,    9,    2, ...,    0,    0,    0],
       [   1,   15,   51, ...,    0,    0,    0],
       [   8,  231, 1054, ...,    0,    0,    0],
       ...,
       [ 442,  479,    1, ...,    0,    0,    0],
       [  10,  777,    6, ...,    0,    0,    0],
       [ 120,  523,  368, ...,    0,    0,    0]])>

In [76]:
model = keras.models.Sequential([
                                 tv,
                                 bow,
                                 Dense(100, activation=keras.activations.relu),
                                 Dense(1, activation=keras.activations.sigmoid)
])

In [77]:
model.compile(optimizer="nadam", loss=keras.losses.binary_crossentropy, metrics=[keras.metrics.binary_accuracy])

In [78]:
model.fit(TRAIN_SET, epochs=5, validation_data=VAL_SET)

Epoch 1/5
782/782 [==============================] - 14s 17ms/step - loss: 0.5456 - binary_accuracy: 0.7144 - val_loss: 0.5147 - val_binary_accuracy: 0.7367
Epoch 2/5
782/782 [==============================] - 13s 17ms/step - loss: 0.4950 - binary_accuracy: 0.7528 - val_loss: 0.5042 - val_binary_accuracy: 0.7443
Epoch 3/5
782/782 [==============================] - 14s 18ms/step - loss: 0.4777 - binary_accuracy: 0.7653 - val_loss: 0.5020 - val_binary_accuracy: 0.7477
Epoch 4/5
782/782 [==============================] - 14s 18ms/step - loss: 0.4555 - binary_accuracy: 0.7812 - val_loss: 0.5034 - val_binary_accuracy: 0.7474
Epoch 5/5
782/782 [==============================] - 14s 18ms/step - loss: 0.4255 - binary_accuracy: 0.8022 - val_loss: 0.5097 - val_binary_accuracy: 0.7444


In [74]:
model = keras.models.Sequential([
    tv,
    bow,
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(TRAIN_SET, epochs=5, validation_data=VAL_SET)

Epoch 1/5
782/782 [==============================] - 15s 19ms/step - loss: 0.5475 - accuracy: 0.7168 - val_loss: 0.5159 - val_accuracy: 0.7367
Epoch 2/5
782/782 [==============================] - 14s 18ms/step - loss: 0.4971 - accuracy: 0.7512 - val_loss: 0.5047 - val_accuracy: 0.7435
Epoch 3/5
782/782 [==============================] - 14s 17ms/step - loss: 0.4813 - accuracy: 0.7626 - val_loss: 0.5022 - val_accuracy: 0.7463
Epoch 4/5
782/782 [==============================] - 14s 18ms/step - loss: 0.4608 - accuracy: 0.7766 - val_loss: 0.5035 - val_accuracy: 0.7451
Epoch 5/5
782/782 [==============================] - 14s 18ms/step - loss: 0.4335 - accuracy: 0.7962 - val_loss: 0.5105 - val_accuracy: 0.7411
